Работа с данными о сообществах в США. Описание датасета:

http://archive.ics.uci.edu/ml/datasets/communities+and+crime

Датасет на кэггле (в формате .csv):

https://www.kaggle.com/kkanda/communities%20and%20crime%20unnormalized%20data%20set

Будем предсказывать количество насильственных преступлений относительно численности населения.

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import warnings

warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('crimedata.csv', na_values=["?"])

# оставим лишь нужные колонки
requiredColumns = [5, 6] + list(range(11,26)) + list(range(32, 103)) + [145]
data = data[data.columns[requiredColumns]]

# некоторые значения целевой переменной пропущены
X = data.loc[data['ViolentCrimesPerPop'].notnull(), :].drop('ViolentCrimesPerPop', axis=1)
y = data['ViolentCrimesPerPop'][X.index]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### 1 Baseline

Обучим линейную регрессию и выведем качество по метрике MSE на обучающей и тестовой выборке.

In [5]:
lr = LinearRegression().fit(X_train,y_train)
print ("Train: {}".format(mean_squared_error(y_train, lr.predict(X_train))))
print ("Test: {}".format(mean_squared_error(y_test, lr.predict(X_test))))


Train: 119935.90613769472
Test: 206978.8843810468


В качестве метода регуляризации используем Ridge ($L_2$-регуляризация).

In [6]:
ridge = Ridge(5.0).fit(X_train,y_train)
print ("Train: {}".format(mean_squared_error(y_train, ridge.predict(X_train))))
print ("Test: {}".format(mean_squared_error(y_test, ridge.predict(X_test))))

Train: 120349.55028715712
Test: 206958.2239545305


### 2 Scaling

Попробуем MinMaxScaler.

In [7]:
sc = MinMaxScaler()
X_train_scaled = pd.DataFrame(data=sc.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(data=sc.transform(X_test), columns=X_test.columns)

Обучения линейной регресии на масштабированных признаках и ошибка на обучающей и тестовой выборке

In [8]:
lr_scaled = LinearRegression().fit(X_train_scaled, y_train)
print('Train {}'.format(mean_squared_error(y_train, lr_scaled.predict(X_train_scaled))))
print('Test {}'.format(mean_squared_error(y_test, lr_scaled.predict(X_test_scaled))))

Train 119943.33803419732
Test 206815.04949143284


Аналогично с добавлением Ridge регуляризации

In [9]:
ridge = Ridge(5.0).fit(X_train_scaled, y_train)
print('Train {}'.format(mean_squared_error(y_train, ridge.predict(X_train_scaled))))
print('Test {}'.format(mean_squared_error(y_test, ridge.predict(X_test_scaled))))

Train 131185.03859170148
Test 171004.31076565202


Ошибка на тестовой выборке возросла, но на тестовой значительно уменьшилась - результат регуляризации

### 3. High/low variance

In [10]:
features_variance = X_train_scaled.var().sort_values(ascending=False)
features_variance.head()

pctUrban          0.197731
RentHighQ         0.063005
MedYrHousBuilt    0.054831
OwnOccHiQuart     0.048807
MedRent           0.046863
dtype: float64

Попробуем удалить признаки с самой низкой дисперсией и посмотреть, как изменится качество.

In [11]:
from sklearn.feature_selection import VarianceThreshold

In [12]:
# можно убрать все признаки, дисперсия которых меньше заданного значения
vs_transformer = VarianceThreshold(0.01)

X_train_var = pd.DataFrame(data=vs_transformer.fit_transform(X_train_scaled), columns=X_train_scaled.columns[vs_transformer.get_support()])
X_test_var = pd.DataFrame(data=vs_transformer.transform(X_test_scaled), columns=X_test_scaled.columns[vs_transformer.get_support()])

X_train_var.shape

(1495, 76)

In [13]:
lr = LinearRegression().fit(X_train_var,y_train)
print ("Train: {}".format(mean_squared_error(y_train, lr.predict(X_train_var))))
print ("Test: {}".format(mean_squared_error(y_test, lr.predict(X_test_var))))

Train: 125706.38916046257
Test: 149123.25580684407


Ошибка уменьшилась

In [14]:
ridge = Ridge(5.0).fit(X_train_var,y_train)
print ("Train: {}".format(mean_squared_error(y_train, ridge.predict(X_train_var))))
print ("Test: {}".format(mean_squared_error(y_test, ridge.predict(X_test_var))))

Train: 136186.7830145162
Test: 152046.21566890884


Ошибка модели возросла, оставим предыдущий вариант

### 4 Correlation

Можно выбрать k признаков, которые дают наиболее высокие значения корреляции с целевой переменной.

In [15]:
from sklearn.feature_selection import SelectKBest, f_regression

In [16]:
# Выбираем 15 лучших признаков
sb = SelectKBest(f_regression, k=15)

X_train_kbest = pd.DataFrame(data=sb.fit_transform(X_train_var, y_train), columns=X_train_var.columns[sb.get_support()])
X_test_kbest = pd.DataFrame(data=sb.transform(X_test_var), columns=X_test_var.columns[sb.get_support()])

In [17]:
lr = LinearRegression().fit(X_train_kbest,y_train)
print ("Train: {}".format(mean_squared_error(y_train, lr.predict(X_train_kbest))))
print ("Test: {}".format(mean_squared_error(y_test, lr.predict(X_test_kbest))))

Train: 147378.18578795987
Test: 156005.7803589241


In [18]:
ridge = Ridge(5.0).fit(X_train_kbest,y_train)
print ("Train: {}".format(mean_squared_error(y_train, ridge.predict(X_train_kbest))))
print ("Test: {}".format(mean_squared_error(y_test, ridge.predict(X_test_kbest))))

Train: 158023.1329983349
Test: 166680.0708593139


А можно выбрать самые значимые признаки с точки зрения регрессии с $L_1$-регуляризацией.

In [19]:
from sklearn.feature_selection import SelectFromModel

In [20]:
lasso = Lasso(5.0)
l1_select = SelectFromModel(lasso)

X_train_l1 = pd.DataFrame(data=l1_select.fit_transform(X_train_var, y_train), columns=X_train_var.columns[l1_select.get_support()])
X_test_l1 = pd.DataFrame(data=l1_select.transform(X_test_var), columns=X_test_var.columns[l1_select.get_support()])

X_train_l1.shape

(1495, 12)

In [22]:
lr = LinearRegression().fit(X_train_l1,y_train)
print ("Train: {}".format(mean_squared_error(y_train, lr.predict(X_train_l1))))
print ("Test: {}".format(mean_squared_error(y_test, lr.predict(X_test_l1))))

ridge = Ridge(5.0).fit(X_train_l1,y_train)
print ("Train: {}".format(mean_squared_error(y_train, ridge.predict(X_train_l1))))
print ("Test: {}".format(mean_squared_error(y_test, ridge.predict(X_test_l1))))

Train: 140757.45879349473
Test: 153086.92726760302
Train: 143263.16845636506
Test: 157553.36533174032


### 5 Pipeline

Сделаем все вышеописанное сразу через pipeline

In [23]:
from sklearn.pipeline import Pipeline


pipe = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('variance', VarianceThreshold(0.01)),
    ('selection', SelectFromModel(Lasso(5.0))),
    ('regressor', Ridge(5.0))
])

pipe.fit(X_train, y_train)

pipe.named_steps

{'scaler': MinMaxScaler(),
 'variance': VarianceThreshold(threshold=0.01),
 'selection': SelectFromModel(estimator=Lasso(alpha=5.0)),
 'regressor': Ridge(alpha=5.0)}

In [24]:
print ("Train: {}".format(mean_squared_error(y_train, pipe.predict(X_train))))
print ("Test: {}".format(mean_squared_error(y_test, pipe.predict(X_test))))

Train: 143263.16845636506
Test: 157553.36533174032


Зададим сетку перебора гиперпараметров с помощью `GridSearch`:

In [25]:
pipe.get_params()

{'memory': None,
 'steps': [('scaler', MinMaxScaler()),
  ('variance', VarianceThreshold(threshold=0.01)),
  ('selection', SelectFromModel(estimator=Lasso(alpha=5.0))),
  ('regressor', Ridge(alpha=5.0))],
 'verbose': False,
 'scaler': MinMaxScaler(),
 'variance': VarianceThreshold(threshold=0.01),
 'selection': SelectFromModel(estimator=Lasso(alpha=5.0)),
 'regressor': Ridge(alpha=5.0),
 'scaler__clip': False,
 'scaler__copy': True,
 'scaler__feature_range': (0, 1),
 'variance__threshold': 0.01,
 'selection__estimator__alpha': 5.0,
 'selection__estimator__copy_X': True,
 'selection__estimator__fit_intercept': True,
 'selection__estimator__max_iter': 1000,
 'selection__estimator__positive': False,
 'selection__estimator__precompute': False,
 'selection__estimator__random_state': None,
 'selection__estimator__selection': 'cyclic',
 'selection__estimator__tol': 0.0001,
 'selection__estimator__warm_start': False,
 'selection__estimator': Lasso(alpha=5.0),
 'selection__importance_getter': '

In [26]:
param_grid = {
    'variance__threshold': [0.005, 0.0075, 0.009, 0.01, 0.011, 0.012],
    'selection__estimator__alpha': [0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0],
    'regressor__alpha': [0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0]
}
grid_search = GridSearchCV(pipe, param_grid, cv=5)

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('variance',
                                        VarianceThreshold(threshold=0.01)),
                                       ('selection',
                                        SelectFromModel(estimator=Lasso(alpha=5.0))),
                                       ('regressor', Ridge(alpha=5.0))]),
             param_grid={'regressor__alpha': [0.1, 0.5, 1.0, 1.5, 2.0, 5.0,
                                              10.0],
                         'selection__estimator__alpha': [0.1, 0.5, 1.0, 1.5,
                                                         2.0, 5.0, 10.0],
                         'variance__threshold': [0.005, 0.0075, 0.009, 0.01,
                                                 0.011, 0.012]})

In [27]:
pipe_best = grid_search.best_estimator_
pipe_best.named_steps

{'scaler': MinMaxScaler(),
 'variance': VarianceThreshold(threshold=0.01),
 'selection': SelectFromModel(estimator=Lasso(alpha=0.1)),
 'regressor': Ridge(alpha=0.5)}

In [28]:
pipe_best.fit(X_train, y_train)
print ("Train: {}".format(mean_squared_error(y_train, pipe_best.predict(X_train))))
print ("Test: {}".format(mean_squared_error(y_test, pipe_best.predict(X_test))))

Train: 128441.16453337153
Test: 147186.92249142198


Получили лучшие гиперпараметры из заданных с меньшей ошибкой модели